In [11]:
import re

In [1]:
import pandas as pd

# Pandas .gz dosyalarını otomatik tanır
# low_memory=False, büyük dosyalarda veri tipi uyarısını engeller
df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv.gz', compression='gzip', low_memory=False)

# İlk 5 satıra bakalım
print(df.head())

# Veri setinin boyutunu görelim (Satır, Sütun)
print(df.shape)

         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months   
3  66310712        NaN    35000.0      35000.0          35000.0   60 months   
4  68476807        NaN    10400.0      10400.0          10400.0   60 months   

   int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0     13.99       123.03     C        C4  ...                            NaN   
1     11.99       820.28     C        C1  ...                            NaN   
2     10.78       432.66     B        B4  ...                            NaN   
3     14.85       829.90     C        C5  ...                            NaN   
4     22.45       289.91     F        F1  ...                            NaN   

  hardship_last_payment_amount disbursement_

In [2]:
# Hangi kredi durumları var ve kaçar tane?
print(df['loan_status'].value_counts(dropna=False))

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
NaN                                                         33
Name: count, dtype: int64


In [3]:
# 1. Her sütundaki eksik veri (NaN) oranını hesapla
missing_fractions = df.isnull().mean().sort_values(ascending=False)

# 2. En çok eksiği olan ilk 10 sütunu görelim
print("En çok eksik veri içeren sütunlar:\n", missing_fractions.head(10))

# 3. %30'dan fazla verisi eksik olan sütunları silelim
drop_list = sorted(list(missing_fractions[missing_fractions > 0.30].index))
print(f"\nToplam {len(drop_list)} adet sütun siliniyor...")

df.drop(labels=drop_list, axis=1, inplace=True)

# Yeni boyuta bakalım
print("Yeni Boyut:", df.shape)

En çok eksik veri içeren sütunlar:
 member_id                                     1.000000
orig_projected_additional_accrued_interest    0.996173
hardship_reason                               0.995171
hardship_payoff_balance_amount                0.995171
hardship_last_payment_amount                  0.995171
payment_plan_start_date                       0.995171
hardship_type                                 0.995171
hardship_status                               0.995171
hardship_start_date                           0.995171
deferral_term                                 0.995171
dtype: float64

Toplam 58 adet sütun siliniyor...
Yeni Boyut: (2260701, 93)


In [4]:
# Sadece 'Fully Paid' ve 'Charged Off' olanları alalım
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

# Hedef değişkeni sayısal yapalım (Batık: 1, Ödenmiş: 0)
# Bu bizim modelin tahmin edeceği "y" değeri olacak.
mapping_dict = {'Charged Off': 1, 'Fully Paid': 0}
df['target'] = df['loan_status'].map(mapping_dict)

# Eskisine ihtiyacımız kalmadı
df.drop('loan_status', axis=1, inplace=True)

print("Filtreleme sonrası boyut:", df.shape)
print("Hedef Dağılımı:\n", df['target'].value_counts())

Filtreleme sonrası boyut: (1345310, 93)
Hedef Dağılımı:
 target
0    1076751
1     268559
Name: count, dtype: int64


In [5]:
# Tüm sütun isimlerini liste olarak yazdır
print(df.columns.tolist())

['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_

In [6]:
# Bu sütunlar kredi verildikten sonra oluşur (Gelecek Bilgisi)
leakage_cols = [
    'recoveries', 'collection_recovery_fee', 'total_pymnt', 'total_pymnt_inv',
    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'last_pymnt_d',
    'last_pymnt_amnt', 'last_credit_pull_d', 'out_prncp', 'out_prncp_inv'
]

# Ayrıca gereksiz (ID, Başlık, vb.) veya tekerrür eden sütunlar
irrelevant_cols = [
    'id', 'url', 'zip_code', 'title', 'emp_title', 'policy_code'
]

# Listeleri birleştirip silelim (Hata almamak için sadece var olanları sil diyoruz)
cols_to_drop = leakage_cols + irrelevant_cols
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True, errors='ignore')

print("Sızıntı temizliği sonrası boyut:", df.shape)

Sızıntı temizliği sonrası boyut: (1345310, 75)


In [8]:
# 1. 'term' sütunundaki " months" yazısını atıp sayıya çevirelim
# Boşlukları temizle ve integer yap
df['term'] = df['term'].str.replace(' months', '').str.strip().astype(float)

# 2. 'emp_length' (Çalışma yılı) sütununu temizleyelim
# "< 1 year" -> 0, "10+ years" -> 10, diğerleri sayı
# NaN değerleri şimdilik 0 kabul edelim (veya ortalama ile doldurabiliriz ama 0 güvenli)
df['emp_length'] = df['emp_length'].str.replace('< 1 year', '0')
df['emp_length'] = df['emp_length'].str.replace('10+ years', '10')
df['emp_length'] = df['emp_length'].str.replace(' years', '')
df['emp_length'] = df['emp_length'].str.replace(' year', '')
# NaN olanları 0 yapıp sayıya çevirelim
df['emp_length'] = df['emp_length'].fillna(0).astype(float)

# Kontrol edelim
print(df[['term', 'emp_length']].head())
print(df[['term', 'emp_length']].dtypes)

   term  emp_length
0  36.0        10.0
1  36.0        10.0
2  60.0        10.0
4  60.0         3.0
5  36.0         4.0
term          float64
emp_length    float64
dtype: object


In [9]:
# 1. Sayısal sütunları bul ve boşlukları MEDYAN ile doldur
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
# target sütununu hariç tutalım (gerçi onda boşluk yok ama garanti olsun)
num_cols = [col for col in num_cols if col != 'target']

print("Sayısal boşluklar dolduruluyor...")
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# 2. Kategorik (Object) sütunları bul ve boşlukları 'Unknown' ile doldur
cat_cols = df.select_dtypes(include=['object']).columns

print("Kategorik boşluklar dolduruluyor...")
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0]) # En sık geçen değerle doldur

# Son kontrol: Hiç boşluk kaldı mı?
print("Kalan toplam boş hücre sayısı:", df.isnull().sum().sum())

Sayısal boşluklar dolduruluyor...
Kategorik boşluklar dolduruluyor...
Kalan toplam boş hücre sayısı: 0


In [10]:
print(df[cat_cols].nunique().sort_values(ascending=False))

earliest_cr_line        739
issue_d                 139
addr_state               51
sub_grade                35
purpose                  14
grade                     7
home_ownership            6
verification_status       3
application_type          2
initial_list_status       2
debt_settlement_flag      2
disbursement_method       2
pymnt_plan                1
hardship_flag             1
dtype: int64


In [12]:
# 1. 50'den fazla çeşidi olan karmaşık sütunları listeden bulup silelim
# (Senin listende earliest_cr_line, issue_d ve addr_state gidecek)
cat_cols = df.select_dtypes(include=['object']).columns
high_card_cols = [col for col in cat_cols if df[col].nunique() > 50]

print(f"Karmaşıklık nedeniyle silinen sütunlar: {high_card_cols}")
df.drop(columns=high_card_cols, inplace=True)

# 2. Kalan kategorik verileri (Grade, Purpose vb.) One-Hot Encoding ile 0-1 yapalım
# drop_first=True diyerek tuzağa düşmeyi engelliyoruz (A sütunu varsa B'ye gerek yok mantığı)
df = pd.get_dummies(df, drop_first=True)

# 3. Sütun isimlerindeki özel karakterleri temizleyelim (Bazı modeller hata verebilir)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

print("\nEncoding Sonrası Yeni Boyut:", df.shape)
# Muhtemelen (1.3 Milyon, 100-120 arası) bir şey çıkacak.

Karmaşıklık nedeniyle silinen sütunlar: ['issue_d', 'addr_state', 'earliest_cr_line']

Encoding Sonrası Yeni Boyut: (1345310, 125)


In [13]:
# Verimiz temizlendi ve encode edildi.
# Şimdi bunu "Checkpoint" olarak kaydedelim.
# index=False diyerek gereksiz satır numaralarını almayalım.
print("Final verisi kaydediliyor... (Bu işlem 1-2 dakika sürebilir)")
df.to_csv('../data/final_processed_data.csv', index=False)
print("✅ Veri başarıyla kaydedildi!")

Final verisi kaydediliyor... (Bu işlem 1-2 dakika sürebilir)
✅ Veri başarıyla kaydedildi!
